## Задание на тексты

1) Скачать fetch_20newsgroups из sklearn.

2) Написать функцию предобработки, которая лемматизирует их (для англ это nltk/spacy, пример nltk - http://textminingonline.com/dive-into-nltk-part-iv-stemming-and-lemmatization)

3) Сравнить качество CountVectorizer, TfIdfVectorizer, HashingVectorizer (1-2 значение размера хэша). Классификатор выберете любой, можно логистическую регрессию или бустинг.

4) Написать функцию, которая делает из предложения один вектор - взвешенные word2vec представления слов по их tf-idf весам.

5) В качестве признаков взять резуьтат функции 4 и посмотреть качество/сравнить со взвешенным по среднему а не tf-idf.

In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
#cat = ['talk.religion.misc','soc.religion.christian']
#ds_train=datasets.load_files('/Users/a1/20news-bydate-train',categories = cat, shuffle=True, random_state=1)

In [3]:
cat = ['talk.religion.misc','soc.religion.christian'] # Выбираем две категории для классификации
# Данные для обучения
ds_train = fetch_20newsgroups(subset='train', categories = cat, shuffle=True, random_state=1
                                  ,remove=('headers', 'footers', 'quotes'))
# Данне для проверки
ds_test = fetch_20newsgroups(subset='test', categories = cat, shuffle=True, random_state=1
                                  ,remove=('headers', 'footers', 'quotes'))

In [4]:
#приведение к нижнему регистру
def get_lower(text):
    return text.lower()

In [5]:
#удаление пунктуации
def remove_punctuation(data):
    exclude = r'[!?,.:;()"*/\\_]'
    s = re.sub(exclude, ' ',data)
    s = re.sub('-', '', s)
    return ' '.join(s.split())

In [6]:
#приведение англ. слов к нормальной форме
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
NORMAL_FORM = {}

# сначала научимся работать со словами
def get_word_normal_form(word):
    if word not in NORMAL_FORM:
        NORMAL_FORM[word] = st.stem(word)
    return NORMAL_FORM[word]

# а затем с текстом
def get_sentence_normal_form(sentence):
    return ' '.join(get_word_normal_form(word) for word in sentence.split())

Обрабатываем обучающие данные

In [7]:
import re
n = len(ds_train.data)
for i in range(n):
    ds_train.data[i] = get_lower(str(ds_train.data[i]))
    ds_train.data[i] = remove_punctuation(str(ds_train.data[i]))
    ds_train.data[i] = get_sentence_normal_form(str(ds_train.data[i]))

In [8]:
ds_train.data[1]
#ds_train.target[5]

"i don't see the effort to equ salv with parad rath i see imply the fact that on thos who ar sav may ent parad"

Обрабатываем тестовые файлы

In [9]:
n = len(ds_test.data)
for i in range(n):
    ds_test.data[i] = remove_punctuation(str(ds_test.data[i]))
    ds_test.data[i] = get_sentence_normal_form(str(ds_test.data[i]))

In [10]:
ds_test.data[1]

"from dhammers@pacific david hammersl how do you mormon reconcil the ide of etern marry with christ's stat that in the resurrect peopl wil neith marry nor be giv in marry luk chapt 20 footnot in som bibl ref thi vers to the book of tobit tobit is in the septuagint goodspee publ it in a book cal the apocryph most any bookst wil hav thi at any rat the jew of christ's day had thi book it is a story most cent around the son of tobit who was nam tobia ther was a young lady sarah who had ent the brid chamb with sev broth in success the broth al died in the chamb bef consum the marry tobia was entitl to hav sarah for his wif 3 17 becaus tobia was her on rel and she was destin for [tobias] from the begin 6 17 tobia took her to wif and was abl to consum the marry the sev husband would not hav her as a partn in heav that doe not elimin tobia her eigh husband tobit is a fun and interest story to read it's kind of a myth rom it's a littl short than esth the lds also hav scriptures that parallel an

CountVectorizer

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
count_vect.fit(ds_train.data)
ds_train_counts = count_vect.transform(ds_train.data)
ds_train_counts.shape

(976, 9937)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
ds_test_counts = count_vect.transform(ds_test.data)
ds_test_counts.shape

(649, 9937)

In [13]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(ds_train_counts, ds_train.target)
print(model)
#make predictions
expected = ds_test.target
predicted = model.predict(ds_test_counts)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
             precision    recall  f1-score   support

          0       0.79      0.78      0.78       398
          1       0.66      0.66      0.66       251

avg / total       0.74      0.73      0.74       649

[[311  87]
 [ 85 166]]


TfidfVectorizer

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
ds_train_tf = vectorizer.fit_transform(ds_train.data)
ds_train_tf.shape

(976, 9937)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
ds_test_tf = vectorizer.transform(ds_test.data)
ds_test_tf.shape

(649, 9937)

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
clf = MultinomialNB(alpha=.01)
clf.fit(ds_train_tf, ds_train.target)
pred = clf.predict(ds_test_tf)
metrics.f1_score(ds_test.target, pred, average='macro')

0.72321875796059842

In [17]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(ds_train_tf, ds_train.target)
print(model)
# make predictions
expected = ds_test.target
predicted = model.predict(ds_test_tf)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
             precision    recall  f1-score   support

          0       0.72      0.91      0.80       398
          1       0.76      0.43      0.55       251

avg / total       0.73      0.73      0.71       649

[[363  35]
 [142 109]]


HashingVectorizer

In [18]:
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer()
ds_train_hv=hv.fit_transform(ds_train.data)
ds_train_hv.shape

(976, 1048576)

In [19]:
from sklearn.feature_extraction.text import HashingVectorizer
ds_test_hv=hv.transform(ds_test.data)
ds_test_hv.shape

(649, 1048576)

In [20]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(ds_train_hv, ds_train.target)
print(model)
# make predictions
expected = ds_test.target
predicted = model.predict(ds_test_hv)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
             precision    recall  f1-score   support

          0       0.70      0.90      0.79       398
          1       0.71      0.39      0.51       251

avg / total       0.71      0.70      0.68       649

[[358  40]
 [152  99]]


## word2vec

In [21]:
import gensim
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import word2vec
import numpy as np

In [22]:
data_w2v = list(sentence.split() for sentence in ds_train.data)

In [23]:
#w2v_model = KeyedVectors.load_word2vec_format('/Users/a1/DA/base/GoogleNews-vectors-negative300.bin', binary=True)
#w2v_model = KeyedVectors.load_word2vec_format(data_w2v, binary=True)
w2v_model = word2vec.Word2Vec(data_w2v, window=5, workers=4)

In [24]:
w2v_model.most_similar('can')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('should', 0.9904413819313049),
 ('may', 0.9839595556259155),
 ('if', 0.9765948057174683),
 ('would', 0.9764683842658997),
 ('do', 0.9711072444915771),
 ('think', 0.9689772725105286),
 ('see', 0.9684304594993591),
 ("don't", 0.9643958806991577),
 ('am', 0.9628705978393555),
 ('wil', 0.9605115056037903)]

In [25]:
w2v_model.most_similar(positive=['he', 'man'], negative=['she'])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('jes', 0.9331137537956238),
 ('god', 0.896336555480957),
 ('his', 0.8932311534881592),
 ('christ', 0.879121720790863),
 ('himself', 0.8612052798271179),
 ('fath', 0.8582339286804199),
 ('son', 0.8419334888458252),
 ('sin', 0.8396385312080383),
 ('lord', 0.8391227126121521),
 ("father's", 0.8309696912765503)]

### средневзвешенная

In [26]:
def get_embedding(sentence):
    words = list(filter(lambda x: x in w2v_model, sentence.split()))
    if len(words) == 0:
        return None
    vectors = list(map(lambda x: w2v_model[x], words))
    sen_embedding = np.mean(vectors, axis=0) #max,median,min...
    sen_embedding /= np.linalg.norm(sen_embedding, axis=0)
    return sen_embedding

In [27]:
ds_train.data[1]

"i don't see the effort to equ salv with parad rath i see imply the fact that on thos who ar sav may ent parad"

In [28]:
data_w2v[1]

['i',
 "don't",
 'see',
 'the',
 'effort',
 'to',
 'equ',
 'salv',
 'with',
 'parad',
 'rath',
 'i',
 'see',
 'imply',
 'the',
 'fact',
 'that',
 'on',
 'thos',
 'who',
 'ar',
 'sav',
 'may',
 'ent',
 'parad']

In [29]:
#get_embedding(data_w2v[1])
get_embedding(str(ds_train.data[1]))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


array([  9.31517407e-02,   2.66948715e-02,  -7.80700594e-02,
        -7.55531481e-03,   2.37862542e-01,   1.01591842e-02,
         7.13977292e-02,  -1.67431254e-02,  -8.62257630e-02,
        -1.09392866e-01,   8.49180296e-02,   1.94837525e-01,
         1.35864586e-01,  -3.91463563e-02,  -1.86599605e-02,
        -1.26634434e-01,  -2.02136770e-01,   3.63745540e-02,
         1.37989834e-01,   1.58018619e-01,   5.66254295e-02,
        -8.59327316e-02,   1.29342541e-01,  -3.04575581e-02,
        -3.40347365e-02,   1.09401196e-02,   7.19777048e-02,
         2.15591509e-02,   2.01317333e-02,  -7.02017769e-02,
         6.56886846e-02,   5.69463633e-02,  -1.09685689e-01,
        -1.07391357e-01,  -1.12998508e-01,   3.60714421e-02,
        -5.68903759e-02,   4.16043364e-02,   1.09745629e-01,
        -2.08835423e-04,   6.61634188e-03,   1.66102499e-02,
         8.47644284e-02,  -1.39446659e-02,   1.48966789e-01,
        -5.57388365e-02,   1.01096697e-01,  -1.45959884e-01,
        -1.88850373e-01,

In [30]:
#w2v_model.save("file.txt")

### взвешенные word2vec представления слов по их tf-idf весам

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
def tfidf_embedding(data, sentence):
    vectors=[]
    vectorizer = TfidfVectorizer()
    tfidf_data =  vectorizer.fit_transform(data)
    vec=vectorizer.get_feature_names()
    
    words = list(filter(lambda x: x in w2v_model, sentence.split()))
    if len(words) == 0:
        return None

    for word in words:
            try:
                vectors.append(tfidf_data[data.index(sentence), vec.index(word)] * w2v_model[word])
            except KeyError:
                pass
    
    sen_embedding = np.mean(vectors, axis=0) #max,median,min...
    sen_embedding /= np.linalg.norm(sen_embedding, axis=0)
    return sen_embedding

In [33]:
#почему то не попадают местоимения((
tfidf_embedding(ds_train.data,str(ds_train.data[1]))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys


ValueError: 'i' is not in list